<a href="https://colab.research.google.com/github/ymarega/INO_IN_EP/blob/main/Optimisation_Hyper_Param.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install tensorflow==2.12.0

In [27]:
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [20]:
def features_creation(df,variable):

    df_copy = df.copy()

    # Les colonnes de Jour J
    df_copy['V0_t-1'] = df_copy[variable].shift(1)
    df_copy['V0_t-2'] = df_copy[variable].shift(2)
    df_copy['V0_t-3'] = df_copy[variable].shift(3)

    # Les colonnes de Jour J-1
    df_copy['V-1_t+2'] = df_copy[variable].shift(4*24-2)
    df_copy['V-1_t+1'] = df_copy[variable].shift(4*24-1)
    df_copy['V-1_t']   = df_copy[variable].shift(4*24)
    df_copy['V-1_t-1'] = df_copy[variable].shift(4*24+1)
    df_copy['V-1_t-2'] = df_copy[variable].shift(4*24+2)

    # Les colonnes de Jour J-2
    df_copy['V-2_t+2'] = df_copy[variable].shift(4*24*2-2)
    df_copy['V-2_t+1'] = df_copy[variable].shift(4*24*2-1)
    df_copy['V-2_t']   = df_copy[variable].shift(4*24*2)
    df_copy['V-2_t-1'] = df_copy[variable].shift(4*24*2+1)
    df_copy['V-2_t-2'] = df_copy[variable].shift(4*24*2+2)
    #df_copy.filter(regex='Q|d|t')

    return df_copy


def remplissage_arriere_simple(df, variable):

    df_sans_na = df[variable].bfill()

    return pd.DataFrame(df_sans_na)

In [13]:
debit_dist_p_stesp = pd.read_csv('https://www.dropbox.com/scl/fi/m7l231zeufqp5ybq0u8t0/debit_dist_p_stesp_2019-08-01_08-10-05_2023-12-31_23-59-50_gran10S.csv?rlkey=meedygqbvcossdw4scszx50ic&dl=1')
debit_dist_p_stesp = debit_dist_p_stesp.rename(columns={'debit_stesprit': 'debit_dist_stesp'})


In [16]:
df_debit_dist_p_stesp_filtrer = debit_dist_p_stesp.copy()

###---Mettre les debits superieurs a 80 a NA (On les consideres comme aberrantes, irréalistes dans les conditions normales)
df_debit_dist_p_stesp_filtrer.loc[df_debit_dist_p_stesp_filtrer['debit_dist_stesp'] > 80, 'debit_dist_stesp'] = np.nan

# Conversion de la colonne 'Date' au format datetime
df_debit_dist_p_stesp_filtrer['Date'] = pd.to_datetime(df_debit_dist_p_stesp_filtrer['Date'])

# Rééchantillonnage à un pas de 15 minutes et calcul de la moyenne
df_debit_dist_p_stesp_15mn = df_debit_dist_p_stesp_filtrer.set_index('Date').resample('15T').mean()

In [30]:


df = df_debit_dist_p_stesp_15mn.copy().tail(10000)
variable = 'debit_dist_stesp'
scaler = MinMaxScaler()
fonction_imputation = remplissage_arriere_simple

# Création des features et normalisation
df_sans_na = fonction_imputation(df, variable)
df_trans = features_creation(df_sans_na, variable).dropna()

# Séparation du jeu de données en entraînement et test
train_size_ratio = 0.75
train_size = int(train_size_ratio * len(df_trans))
train, test = df_trans[:train_size], df_trans[train_size:]

# Séparation des variables explicatives et de la cible
X_train, y_train = train.drop(variable, axis=1), train[variable]
X_test, y_test = test.drop(variable, axis=1), test[variable]


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Redimensionnement pour le modèle LSTM
X_train_reshaped = X_train_scaled.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test.shape[0], 1, X_test.shape[1]))

def create_model(units=50, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units, activation=activation, return_sequences=True, input_shape=(1, X_train.shape[1])))
    model.add(LSTM(units, activation=activation, return_sequences=True))
    model.add(LSTM(units, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model


model = KerasRegressor(build_fn=create_model, verbose=0)

param_grid = {
    'units': [50, 100],  # Test avec différentes unités pour les couches
    'activation': ['relu', 'tanh'],  # Test de différentes fonctions d'activation
    'optimizer': ['adam', 'rmsprop'],  # Test de différents optimiseurs
    'epochs': [10, 20],  # Test de différents nombres d'époques
    'batch_size': [32, 64]  # Test de différentes tailles de lots
}

# Scorer qui calcule le RMSE négatif, car GridSearchCV maximise le score
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=rmse_scorer, n_jobs=-1, cv=3)

grid_result = grid.fit(X_train_reshaped, y_train)

# Affichage des meilleurs paramètres et de la meilleure performance (RMSE)
print("Meilleur RMSE: %f en utilisant %s" % (-grid_result.best_score_, grid_result.best_params_))

# Pour évaluer le modèle sur les données de test, utilisez le meilleur modèle trouvé
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test_reshaped)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE sur les données de test: %f" % test_rmse)



<ipython-input-30-fb4d81ab8bbd>:37: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Meilleur RMSE: 2.020323 en utilisant {'activation': 'relu', 'batch_size': 32, 'epochs': 20, 'optimizer': 'adam', 'units': 100}
RMSE sur les données de test: 1.916604


In [29]:
 df_debit_dist_p_stesp_15mn.copy().tail(10000)

,debit_dist_stesp
Date,
2023-09-18 20:00:00,35.027046
2023-09-18 20:15:00,33.051748
2023-09-18 20:30:00,30.545576
2023-09-18 20:45:00,29.934513
2023-09-18 21:00:00,30.956855
...,...
2023-12-31 22:45:00,11.152983
2023-12-31 23:00:00,10.602996
2023-12-31 23:15:00,10.302007
